In [1]:
!pip install datasets
!sudo apt-get install swig
!sudo pip install jamspell
!wget https://github.com/bakwc/JamSpell-models/raw/master/en.tar.gz
!tar -xvf en.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 13.4 MB/s 
     |████████████████████████████████| 182 kB 80.1 MB/s 
     |████████████████████████████████| 213 kB 68.8 MB/s 
     |████████████████████████████████| 132 kB 82.9 MB/s 
     |████████████████████████████████| 127 kB 61.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newl

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import altair as alt



import altair as alt
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import jamspell
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV 

import datasets
from datasets import load_dataset

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [3]:
# cargamos el corpus de texto
dataset = datasets.load_dataset('tweet_eval', 'sentiment')

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45615 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12284 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# Convertimos los datasets en formato dataframe
train_df = pd.DataFrame(dataset['train'])
val_df = pd.DataFrame(dataset['validation'])
test_df = pd.DataFrame(dataset['test'])

In [5]:
# Na check.
train_df.text.isnull().sum()

0

In [6]:
y = train_df.label.copy()

y[y == 0] = 'Negative'
y[y == 1] = 'Neutral'
y[y == 2] = 'Positive'


# Labels desbalanceadas.
counts = y.value_counts(normalize=True).reset_index()
counts.columns = ['target', 'ratio']

alt.Chart(counts).mark_bar().encode(
    x=alt.X('target:N', title='Categorías'),
    y=alt.Y('ratio:Q', scale=alt.Scale(domain=(0, 1)), title='Ratio sobre total'),
    tooltip=['target', 'ratio']
).properties(width=600, height=200)

alt.Chart(...)

In [7]:
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('en.bin')
lemmatizer= WordNetLemmatizer()
punct = ''.join([i for i in string.punctuation if i not in [',','?','!','.']])

In [8]:
# Elaboramos la funcion de limpieza de texto

def hard_clean(x):
  # normalizacion del texto es minusculas.
  x = x.lower()
  x = x.encode('ascii', 'ignore').decode()
  # Removemos las url.
  x = re.sub(r"http\S+", ' ', x)
  # Removemos @usuarios.
  x = re.sub(r'@[\w]+', '', x)
  # Removemos Hashtag
  x = re.sub(r'#[\w]+', '', x)
  # Removemos los signos de puntuacion.
  x = re.sub('[^\w\s]', '', x)
  # Spell check
  x = corrector.FixFragment(x)
  # Removemos palabras con menos de dos caracteres.
  x = ' '.join([w for w in x.split() if len(w)>2])
  # Lematizacion y stop words removal
  x = ' '.join([lemmatizer.lemmatize(word) for word in x.split() if word not in stopwords.words('english')])
  return x

In [9]:
example = train_df.text[45610] + " #hashtag"

In [10]:
hard_clean(example)

'amazing beautiful lady gaga show tonight love girl britney'

In [11]:
train_df['text_c1'] = train_df['text'].apply(hard_clean)

In [12]:
# Vectorizamos las palabras de
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)

train_vectors1 = vectorizer.fit_transform(train_df['text_c1'])

In [13]:
train_vectors1 = vectorizer.fit_transform(train_df['text_c1'])

In [14]:
# Calculamos los pesos de cada clase para darselos al modelo
class_weights = (1 - (train_df['label'].value_counts().sort_index() / len(train_df))).values
class_weights

array([0.8445029 , 0.54679382, 0.60870328])

In [15]:
weights = {}

for idx, i in enumerate(class_weights):
  weights[idx] = i

weights

{0: 0.8445029047462458, 1: 0.5467938178230844, 2: 0.6087032774306698}

In [16]:
# Cargamos el modelo y le proporcionamos los pesos
svm = SVC(class_weight=weights)

In [17]:
# Creamos un grid de parametros para el GridSearch
param_grid = {'C': [0.001,0.1,1], 'gamma': [1,0.1,0.001], 'kernel': ['linear', 'rbf']}

In [18]:
# Inicializamos gridsearch
grid = GridSearchCV(svm, param_grid)

In [19]:
# Procedemos a entrenarlo
svm1 = grid.fit(train_vectors1, train_df['label'].values.ravel())

In [20]:
svm1

GridSearchCV(estimator=SVC(class_weight={0: 0.8445029047462458,
                                         1: 0.5467938178230844,
                                         2: 0.6087032774306698}),
             param_grid={'C': [0.001, 0.1, 1], 'gamma': [1, 0.1, 0.001],
                         'kernel': ['linear', 'rbf']})

In [21]:
svm1.best_params_

{'C': 1, 'gamma': 1, 'kernel': 'linear'}

In [22]:
test_vectors = vectorizer.transform(val_df['text'])

In [23]:
predictionsc1 = grid.predict(test_vectors)

In [24]:
from sklearn.metrics import classification_report
report = classification_report(val_df['label'], predictionsc1, output_dict=True)

In [25]:
report

{'0': {'precision': 0.5598290598290598,
  'recall': 0.4198717948717949,
  'f1-score': 0.4798534798534798,
  'support': 312},
 '1': {'precision': 0.6264150943396226,
  'recall': 0.7640966628308401,
  'f1-score': 0.6884396060134784,
  'support': 869},
 '2': {'precision': 0.7464589235127479,
  'recall': 0.6434676434676435,
  'f1-score': 0.6911475409836065,
  'support': 819},
 'accuracy': 0.661,
 'macro avg': {'precision': 0.6442343592271434,
  'recall': 0.6091453670567595,
  'f1-score': 0.6198135422835216,
  'support': 2000},
 'weighted avg': {'precision': 0.6651856210023696,
  'recall': 0.661,
  'f1-score': 0.657009069702786,
  'support': 2000}}